In [1]:
RAW_DATA = r'C:\Users\411792\Desktop\LIFE_TEST_DATA\V6L_LTS-002700_THT3546-47-53_55C.csv'
out_path = r'C:\Users\411792\Desktop\LIFE_TEST_DATA\V6L_LTS-002700_THT3546-47-53_55C.xlsx'

In [2]:
import xlsxwriter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import openpyxl
from openpyxl.chart import BarChart3D,Reference

In [3]:
df = pd.read_csv(RAW_DATA)
df = df.sort_values(['Magnet','Head'])
grouped_df = df.groupby(['Magnet','Head']).first()
df = pd.merge(df, grouped_df, on=['Magnet','Head'] , suffixes=('','_REF'))
df['TTF'] = np.where(df['Head']!=df['Head'].shift(),0,df['TotalStressTime'])/60
df['PCT_CNG'] = (df['Ohms']/df['Ohms_REF'] - 1)*100
df['P/F Status'] = np.where(abs(df['PCT_CNG'])>=20,1,0)
df['P/F Status'] = df.groupby(['Magnet','Head'])['P/F Status'].cumsum()
df['P/F Status'] = np.where(df['Head']!=df['Head'].shift(),0,np.where(df['Head']!=df['Head'].shift(-1),1,np.where(df['P/F Status']>0,1,0)))
df['P/F Flag (10%)'] = np.where(df['Head']!=df['Head'].shift(),0,np.where(df['P/F Status']!=df['P/F Status'].shift(),1,0))
df['F/S'] = np.where((df['P/F Flag (10%)']==1)&(abs(df['PCT_CNG'])>=20),'F','S')
df['SBR'] = np.where(df['Magnet']==15,'THT3546',np.where(df['Head']==0,'THT3553',np.where(df['Head']==3,'THT3553',np.where(df['Head']==4,'THT3553',np.where(df['Head']==7,'THT3553',np.where(df['Magnet']>9,'THT3547',np.where(df['Head']==1,'THT3546',np.where(df['Magnet']==8,'THT3546','THT3547'))))))))
df['Stress(mA)'] = np.where(df['Head']<2,67,np.where(df['Head']<4,68,np.where(df['Head']<6,69,70)))
df['Designs'] = np.where(df['SBR']=='THT3546','LRT1.2 POR',np.where(df['SBR']=='THT3547','LRT1.2 POR','LRT2.0 M1'))
df = df[['HeadID','MagnetTemp','Magnet','Head','Carrier','StressTimeInc','StressCurrentDC','Stress Ohms','TotalStressTime','Ohms','Ohms_REF','TTF','PCT_CNG','P/F Status','P/F Flag (10%)','F/S','SBR','Stress(mA)','Designs']]

dy = df.loc[df['P/F Flag (10%)'] == 1]
table = pd.pivot_table(dy, values='HeadID', index=['F/S','TTF'], columns=['SBR','Stress(mA)'], aggfunc='count', margins=False)

In [4]:
writer = pd.ExcelWriter(out_path, engine='xlsxwriter')
df.to_excel(writer, sheet_name='ALL', index=None)
table.to_excel(writer, sheet_name='SUM')
dy.to_excel(writer, sheet_name='TTF')
writer.save()